In [1]:
!pip install -q transformers peft bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.9 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

# Path to final checkpoint
checkpoint_dir = "/content/drive/Shareddrives/DATA266_Project/Checkpoints/mistral_spider_lora/checkpoint-1311"

# Load base Mistral model with quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    quantization_config=bnb_config,
    device_map="auto"
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
tokenizer.pad_token = tokenizer.eos_token  # set pad token

# Load PEFT adapter from saved checkpoint
model = PeftModel.from_pretrained(base_model, checkpoint_dir)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [3]:
# Example: test prompt from Spider dev set
instruction = """### Instruction:
Generate an SQL query for the following question based on the database schema.

### Question:
How many heads of the departments are older than 56?

### Schema:
department(Department_ID:number, Name:text, Creation:text, Ranking:number, Budget_in_Billions:number, Num_Employees:number)
head(head_ID:number, name:text, born_state:text, age:number)
management(department_ID:number, head_ID:number, temporary_acting:text)

### SQL:
"""

# Tokenize the prompt
inputs = tokenizer(instruction, return_tensors="pt").to(model.device)

# Generate output from model
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,         # limit generation length
        temperature=0.0,            # deterministic output
        do_sample=False,            # no sampling
        pad_token_id=tokenizer.eos_token_id
    )

# Decode and print result
generated_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_sql)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


### Instruction:
Generate an SQL query for the following question based on the database schema.

### Question:
How many heads of the departments are older than 56?

### Schema:
department(Department_ID:number, Name:text, Creation:text, Ranking:number, Budget_in_Billions:number, Num_Employees:number)
head(head_ID:number, name:text, born_state:text, age:number)
management(department_ID:number, head_ID:number, temporary_acting:text)

### SQL:
SELECT count(*) FROM head WHERE age  >  56;

### Schema:
department(Department_ID:number, Name:text, Creation:text, Ranking:number, Budget_in_Billions:number, Num_Employees:number)
head(head_ID:number, name:text, born_state:text, age:number)
management(department_ID:number, head_ID


In [4]:
# Extract only the SQL after "### SQL:"
sql_start = generated_sql.split("### SQL:")[-1].strip()

# Stop generation if schema is repeated (model keeps talking)
sql_cleaned = sql_start.split("###")[0].strip()

print("Generated SQL:\n", sql_cleaned)

Generated SQL:
 SELECT count(*) FROM head WHERE age  >  56;


In [5]:
# Complex example from Spider (e.g., car manufacturers in Europe with filters)
instruction = """### Instruction:
Generate an SQL query for the following question based on the database schema.

### Question:
Which countries in Europe have at least 3 car manufacturers?

### Schema:
countries(countryid:number, countryname:text, continent:text)
continents(contid:number, continent:text)
car_makers(id:number, name:text, country:number)

### SQL:
"""

# Tokenize input
inputs = tokenizer(instruction, return_tensors="pt").to(model.device)

# Generate SQL from the model
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.0,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )

# Decode and trim output
generated_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)
sql_text = generated_sql.split("### SQL:")[-1].split("###")[0].strip()

print("Generated SQL:\n", sql_text)

Generated SQL:
 SELECT T1.countryname FROM countries AS T1 JOIN car_makers AS T2 ON T1.countryid  =  T2.country WHERE T1.continent  =  'Europe' GROUP BY T1.countryid HAVING count(*)  >=  3
